## Objective
**Clean and preprocess** loan data for the data science team.
<br>They will create a Credit Risk Model that measures probability of default (not included in this project).

---

Notes from the data science team:
- Convert all currencies to Euro
- Quantify every categorical variable
- When measuring **credit worthiness** be extremely risk-adverse and distrustful of any unavailable data
- Casting directions are provided for each variable


## Import packages

In [2]:
import numpy as np

#Suppresses numpy's default of displaying scientific notation, allow 100 characters per line and decimal places to 2
np.set_printoptions(suppress = True, linewidth = 100, precision = 2) 

## Importing the data

In [6]:
raw_data_np = np.genfromtxt('loan-data.csv', delimiter = ';')
raw_data_np

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 341: invalid start byte

# ❗️❗️❗️

In [12]:
#using a different csv encoding since utf-8 can't decode some values in the csv file
#also dropping the header column and removing all leading and trailing white spaces
raw_data_np = np.genfromtxt('loan-data.csv', delimiter = ';', encoding = 'cp855', skip_header = 1, autostrip = True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

___

## Checking for incomplete data

In [13]:
np.isnan(raw_data_np).sum() #number of NaN values in the dataset

88005

In [14]:
temporary_fill = np.nanmax(raw_data_np) + 1 #creates a value higher than any value in the ds which will be used as a filler
temporary_mean = np.nanmean(raw_data_np, axis = 0) #mean for every column ignoring means

<ipython-input-14-0d0f1d6e721d>:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data_np, axis = 0) #mean for every column


Above warning: mean of empty slice indicates that there are **columns containing all NaN values** or columns containing all **strings**

In [15]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

At this point it would make sense to split this array into two arrays, a numerical and string array.

In [16]:
#Store stats of each column in an array prior to split
temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
                           temporary_mean,
                           np.nanmax(raw_data_np, axis = 0)])

<ipython-input-16-373822180ae4>:2: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
<ipython-input-16-373822180ae4>:4: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis = 0)])


In [18]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

___

## Splitting the Dataset

# 🧐
For convenience we'll split the data into numerical and categorical datasets, the latter will then be numerised based on a column to column basis

### Splitting the columns

In [155]:
#argwhere test weather the argument is != zero
#columns with just strings returns True which is != 0
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze() #squeeze method returns a 1 dimensional array
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [156]:
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

### Re-importing the dataset
With the above constraints

In [157]:
# String dataset
loan_data_strings = np.genfromtxt('loan-data.csv',
                                  delimiter = ';',
                                  skip_header = 1,
                                  autostrip = True,
                                  encoding = "cp855",
                                  usecols = columns_strings,
                                  dtype = str)
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [278]:
# Numeric dataset
loan_data_numeric = np.genfromtxt('loan-data.csv',
                                  delimiter = ';',
                                  skip_header = 1,
                                  autostrip = True,
                                  encoding = "cp855",
                                  usecols = columns_numeric,
                                  filling_values = temporary_fill)
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

### The Names of the Columns

In [280]:
header_full = np.genfromtxt('loan-data.csv',
                             delimiter = ';',
                             #this line skips all rows besides the header, it works because when we loaded raw_data_np, we imported all rows but the header
                             skip_footer = raw_data_np.shape[0],
                             encoding = "cp855",
                             dtype = str)

header_full

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [282]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

In [96]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [283]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

___

## Creating Checkpoints:

# 🔐

In [161]:
#function parameters, file_name, checpoint_header and the data
#creates an npz file from parameters
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + '.npz')
    return(checkpoint_variable)

In [38]:
checkpoint_test = checkpoint('checkpoint-test', header_strings, loan_data_strings) #test function

In [40]:
checkpoint_test['header'] #header should be equal to our header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [162]:
np.array_equal(checkpoint_test['data'], loan_data_strings)

True

___

## Manipulating Strings Columns

In [42]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [163]:
#rename issue_d = issue_date
header_strings[0] = 'issue_date'

In [44]:
loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

### Issue Date (issue_date)

In [45]:
loan_data_strings[:,0]
#Values seem to be all Month-YY format, of the year 2015 (can strip the year, since no additionally unique info)

array(['May-15', '', 'Sep-15', ..., 'Jun-15', 'Apr-15', 'Dec-15'], dtype='<U69')

In [46]:
np.unique(loan_data_strings[:,0])
#All values seem to follow that format, empty space indicates missing values

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

In [164]:
#Dropping the '-15'
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], '-15')

In [165]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')


**Substitute Months for numbers**
- Use less memory and space
- Easy to follow order of the months

In [166]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [167]:
#Use a for loop to assign numbers (the index) to the values of loan_data_strings[:,0]
for i in range(13):
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                      i,
                                      loan_data_strings[:,0])

In [103]:
np.unique(loan_data_strings[:,0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

✅ All months have been substituted with a corresponding number

___

### Loan Status (loan_status)

In [52]:
loan_data_strings[:,1]

array(['Current', 'Current', 'Current', ..., 'Current', 'Current', 'Current'], dtype='<U69')

In [54]:
np.unique(loan_data_strings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [55]:
np.unique(loan_data_strings[:,1]).size

9

# 🧐
There are 9 unique values for loan_status, including empty spaces.
<br>We could assign a different value to each one but our **regression model only cares if the candidate is in a stable financial position**
<br>We will use a dummy variable and split all values into either 'good'(1) or 'bad'(0) loan status.
<br>**1**: Current, Fully Paid, In Grace Period, Issued, Late (16 - 30 days)
<br>**0**: No data, Charged Off, Default, Late (31 - 120 days)

In [168]:
status_bad = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)']) #array with bad status values

In [169]:
#check to see if the value for a given row in loan_status is equal to any value in the status_bad array
#if so, assign it a value of 0, else a value of 1
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad),0,1)

In [106]:
np.unique(loan_data_strings[:,1])

array(['0', '1'], dtype='<U69')

✅ loan_status has been successfully transformed to contain only 1's and 0's indicating a good or bad loan status, respectively
___

### Term (term)

In [60]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [61]:
loan_data_strings[:,2]

array(['36 months', '36 months', '36 months', ..., '36 months', '36 months', '36 months'],
      dtype='<U69')

In [62]:
np.unique(loan_data_strings[:,2])

array(['', '36 months', '60 months'], dtype='<U69')

# 🧐
3 unique values
<br>Can strip the word ' months', it provides no additional info
<br>And change the header accordingly to 'term_months'
<br>As per our instructions, with missing data we assume the worst, we will therefore assign 60 to all empty spaces

In [170]:
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], ' months') #remove the word months
np.unique(loan_data_strings[:,2])

array(['', '36', '60'], dtype='<U69')

In [171]:
header_strings[2] = 'term_months' #change the column name to something more descriptive
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [172]:
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '',
                                  '60',
                                  loan_data_strings[:,2])
np.unique(loan_data_strings[:,2])

array(['36', '60'], dtype='<U69')

✅ We will assume there is a significance to these values and therefore not change them to 1 and 0
___

### Grade (grade) and Sub-Grade (sub_grade)

In [66]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'grade', 'sub_grade', 'verification_status',
       'url', 'addr_state'], dtype='<U19')

In [67]:
np.unique(loan_data_strings[:,3])

array(['', 'A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype='<U69')

In [93]:
np.unique(loan_data_strings[:,4])

array(['5', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

# 🧐
Firstly, both columns have missing values.
<br>The sub_grade is obviously a more detailed breakdown of the grade value. 
<br>Since grade is included in the sub_grade column it goes to reason that we can drop this column as it is redundant. Before we do that though:
<br>Missing values in the sub_grade column will be assigned it's parent grade value, if that value exist, and the number 5, since we assume the worst for our CRM
<br>For example a missing **sub_grade** value with a parent **grade** value of 'B' will be assigned a value of 'B5'

#### Filling 

In [173]:
for i in np.unique(loan_data_strings[:,3])[1:]:                                                       #iterating through the np.unique array fro the 2nd value since the first is an empty space
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i), #where the value in column sub_grade is an empty space
                                      i + '5',                                                        #and the value of the grade column is equal to the current value of the for loop (np.unique array)
                                      loan_data_strings[:,4])                                         #assign the current value of the for loop + the number 5

In [111]:
np.unique(loan_data_strings[:,4])

array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
       'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
       'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69')

We still have missing data, which means there are rows with no grades and no sub_grades

In [174]:
np.unique(loan_data_strings[:,4], return_counts = True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5]))

There are only 9 rows with no values, we could drop these rows as there statistical significance will be low.
However, since we have to assign values to all rows we will assign a new value to the missing elements of 'H1'

In [175]:
#Assign the vale of 'H1' to sub_grade for all remaining rows with no values for Grade and Sub_Grade
loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                  'H1',
                                  loan_data_strings[:,4])
np.unique(loan_data_strings[:,4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

We have no more missing values in sub_grade and can now drop the redundant grade column

#### Removing Grade

In [176]:
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [115]:
loan_data_strings[:,3]

array(['C3', 'A5', 'B5', ..., 'A5', 'D2', 'A4'], dtype='<U69')

Update header strings acordingly

In [177]:
header_strings = np.delete(header_strings, 3) #one dimensional array, no need to indicate axis
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

✅ Grade column removed and header_strings array updated acordingly

#### Converting Sub Grade

In [117]:
np.unique(loan_data_strings[:,3])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

Objective assign values 1 to 36 to sub_grade values A1 to H1 (36 subgrade values in total)
<br>keys = loan_data_strings[:,3]
<br>values = create a list with range 1 to **loan_data_strings[:,3].shape[0] + 1**, which is equal to the length of the array + 1...since arrays are inclusive-exclusive

In [178]:
keys = list(np.unique(loan_data_strings[:,3]))
keys

['A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'D1',
 'D2',
 'D3',
 'D4',
 'D5',
 'E1',
 'E2',
 'E3',
 'E4',
 'E5',
 'F1',
 'F2',
 'F3',
 'F4',
 'F5',
 'G1',
 'G2',
 'G3',
 'G4',
 'G5',
 'H1']

In [179]:
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1))

In [128]:
values

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36]

In [180]:
dict_sub_grade = dict(zip(keys, values)) #zip functions pairs items in iterators together

In [130]:
dict_sub_grade

{'A1': 1,
 'A2': 2,
 'A3': 3,
 'A4': 4,
 'A5': 5,
 'B1': 6,
 'B2': 7,
 'B3': 8,
 'B4': 9,
 'B5': 10,
 'C1': 11,
 'C2': 12,
 'C3': 13,
 'C4': 14,
 'C5': 15,
 'D1': 16,
 'D2': 17,
 'D3': 18,
 'D4': 19,
 'D5': 20,
 'E1': 21,
 'E2': 22,
 'E3': 23,
 'E4': 24,
 'E5': 25,
 'F1': 26,
 'F2': 27,
 'F3': 28,
 'F4': 29,
 'F5': 30,
 'G1': 31,
 'G2': 32,
 'G3': 33,
 'G4': 34,
 'G5': 35,
 'H1': 36}

In [181]:
for i in np.unique(loan_data_strings[:,3]):                        #iterate through array with unique sub_grade values...A1 to H1
    loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, #where the value in the sub_grade column is equal to the current i value, as in the current sub_grade in np.unique
                                      dict_sub_grade[i],           #update the value in the sub_grade column with the value of dict_sub_grade with a key = i
                                      loan_data_strings[:,3])      #else leave value as is

In [134]:
np.unique(loan_data_strings[:,3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

✅ All sub_grade values, A1 to H1 have been numerised to '1' to '36'
___

### Verification Status (verification_status)

In [135]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [136]:
loan_data_strings[:,4]

array(['Verified', 'Source Verified', 'Verified', ..., 'Source Verified', 'Source Verified', ''],
      dtype='<U69')

In [137]:
np.unique(loan_data_strings[:,4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

# 🧐
- 1 = 'Verified' and 'Source Verfied'
- 0 = '' and 'Not Verfied'

In [182]:
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') | (loan_data_strings[:,4] == 'Not Verified'), 0, 1)

In [183]:
np.unique(loan_data_strings[:,4])

array(['0', '1'], dtype='<U69')

✅ verification_status have been successfully numerised to '1' and '0'
___

### URL (url)

In [141]:
loan_data_strings[:,5]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', ...,
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249'], dtype='<U69')

# 🧐
The URL field contains no unique values but for the id. We will therefore strip everything but the id.

In [184]:
np.chararray.strip(loan_data_strings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')

chararray(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'],
          dtype='<U69')

In [185]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], 'https://www.lendingclub.com/browse/loanDetail.action?loan_id=')
loan_data_strings[:,5]

array(['48010226', '57693261', '59432726', ..., '50415990', '46154151', '66055249'], dtype='<U69')

In [144]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [147]:
loan_data_numeric[:,0]

array([48010226., 57693261., 59432726., ..., 50415990., 46154151., 66055249.])

# 🧐
The id in our numeric dataset and the stripped id in our URL column of the strings dataset seems to contain identical information.
<br>The former is a decimal number and the latter a string.
<br>Lets cast both as integers and compare them

In [150]:
loan_data_numeric[:,0].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [151]:
loan_data_strings[:,5].astype(dtype = np.int32)

array([48010226, 57693261, 59432726, ..., 50415990, 46154151, 66055249], dtype=int32)

In [152]:
np.array_equal(loan_data_numeric[:,0].astype(dtype = np.int32), loan_data_strings[:,5].astype(dtype = np.int32))

True

# 🧐
The ID and stripped URL fields are equal.
<br>We can drop this colum in the strings dataset and update the header.

In [186]:
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

In [187]:
loan_data_strings[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [188]:
header_strings[5]

'addr_state'

✅ URL column dropped as it contained no additional information. The ID in the URL can be obtained from the ID column in the dataset.
___

### State Address (state_address)

In [189]:
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'addr_state'], dtype='<U19')

In [190]:
header_strings[5] = 'state_address'
header_strings

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [192]:
loan_data_strings[:,5]

array(['CA', 'NY', 'PA', ..., 'CA', 'OH', 'IL'], dtype='<U69')

In [194]:
np.unique(loan_data_strings[:,5])

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [195]:
np.unique(loan_data_strings[:,5]).size

50

# 🧐
We have 50 values, including an empty space, which means we are missing a state.
<br>The missing state is Iowa(IA) which was purposefully left as a baseline benchmark

In [196]:
np.unique(loan_data_strings[:,5], return_counts = True)

(array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
        'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
        'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
        'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69'),
 array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
          84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
         341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
          17,  216,  148,   49,   27]))

# 🧐
It will make more sense to sort these arrays in descending order according to the number of times a state appears in this column.

In [197]:
state_names, state_counts = np.unique(loan_data_strings[:,5], return_counts = True)

In [198]:
state_names

array(['', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [199]:
state_counts

array([ 500,   26,  119,   74,  220, 1336,  201,  143,   27,   27,  690,  321,   44,  389,  152,
         84,   84,  116,  210,  222,   10,  267,  156,  160,   61,   28,  261,   16,   25,   58,
        341,   57,  130,  777,  312,   83,  108,  320,   40,  107,   24,  143,  758,   74,  242,
         17,  216,  148,   49,   27])

In [201]:
states_count_sorted = np.argsort(-state_counts) #create a descending sorted array
states_count_sorted

array([ 5, 33, 42, 10,  0, 13, 30, 11, 37, 34, 21, 26, 44, 19,  4, 46, 18,  6, 23, 22, 14, 47,  7,
       41, 32,  2, 17, 36, 39, 16, 15, 35, 43,  3, 24, 29, 31, 48, 12, 38, 25,  9,  8, 49,  1, 28,
       40, 45, 27, 20])

In [202]:
state_names[states_count_sorted], state_counts[states_count_sorted] #use states_count_sorted as the index order variable for both the names and counts arrays

(array(['CA', 'NY', 'TX', 'FL', '', 'IL', 'NJ', 'GA', 'PA', 'OH', 'MI', 'NC', 'VA', 'MD', 'AZ',
        'WA', 'MA', 'CO', 'MO', 'MN', 'IN', 'WI', 'CT', 'TN', 'NV', 'AL', 'LA', 'OR', 'SC', 'KY',
        'KS', 'OK', 'UT', 'AR', 'MS', 'NH', 'NM', 'WV', 'HI', 'RI', 'MT', 'DE', 'DC', 'WY', 'AK',
        'NE', 'SD', 'VT', 'ND', 'ME'], dtype='<U69'),
 array([1336,  777,  758,  690,  500,  389,  341,  321,  320,  312,  267,  261,  242,  222,  220,
         216,  210,  201,  160,  156,  152,  148,  143,  143,  130,  119,  116,  108,  107,   84,
          84,   83,   74,   74,   61,   58,   57,   49,   44,   40,   28,   27,   27,   27,   26,
          25,   24,   17,   16,   10]))

# 🧐
There are 500 missing values, which is more than the entries for all but 4 states.
<br>We also have very little data to examine each state individually as outliers in such small datasets can have a big influence on the coefficients for less represented states.
<br>To solve this problem we will group states according to a common characetistic: regions.
<br>First we will assign the value 0 to all empty spaces and not include them in our regions to avoid biassing with variable coefficients.

In [203]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '',
                                  0,
                                  loan_data_strings[:,5])
np.unique(loan_data_strings[:,5])

array(['0', 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN',
       'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
       'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA',
       'VT', 'WA', 'WI', 'WV', 'WY'], dtype='<U69')

In [204]:
#create 4 arrays, one for each region of the USA and assign states accordingly
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

In [205]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])    #checks weather the value in state_address is in the state_west array and assigns a 1 if true, else leave as is if false
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])   #checks weather the value in state_address is in the state_south array and assigns a 2 if true, else leave as is if false
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])   #checks weather the value in state_address is in the state_midwest array and assigns a 3 if true, else leave as is if false
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])   #checks weather the value in state_address is in the state_east array and assigns a 4 if true, else leave as is if false

In [206]:
np.unique(loan_data_strings[:,5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

✅ All states have been grouped according to one of four USA regions.
<br>All empty spaces were assigned the value of 0
___

### Converting to numbers

In [207]:
loan_data_strings #all values, although numbers, are still in string format

array([['5', '1', '36', '13', '1', '1'],
       ['0', '1', '36', '5', '1', '4'],
       ['9', '1', '36', '10', '1', '4'],
       ...,
       ['6', '1', '36', '5', '1', '1'],
       ['4', '1', '36', '17', '1', '3'],
       ['12', '1', '36', '4', '0', '3']], dtype='<U69')

In [209]:
loan_data_strings = loan_data_strings.astype(int)
loan_data_strings

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

✅ All values in data_strings converted to integer numbers
___

# 🔐

## Checkpoint 1: Strings

In [210]:
checkpoint_strings = checkpoint('Checkpoint-Strings', header_strings, loan_data_strings)

In [211]:
checkpoint_strings['header']

array(['issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [212]:
checkpoint_strings['data']

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       ...,
       [ 6,  1, 36,  5,  1,  1],
       [ 4,  1, 36, 17,  1,  3],
       [12,  1, 36,  4,  0,  3]])

In [213]:
np.array_equal(loan_data_strings, checkpoint_strings['data'])

True

✅ Checkpoint for string dataset successfully created and saved
___

## Manipulating Numeric Columns

In [284]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  , 68616520.  , 68616520.  ,     2185.64],
       [46154151.  , 68616520.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  , 68616520.  ,      309.97,      301.9 ]])

In [285]:
np.isnan(loan_data_numeric).sum()

0

# 🧐
Currently no NaN values because we replaced them with the temporary_fill (temporary_fill = np.nanmax(raw_data_np) + 1)
<br>We will check to see which columns contain the temporary_fill value

### Substitute "Filler" Values

In [216]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

### ID (id)

In [217]:
temporary_fill

68616520.0

In [219]:
np.isin(loan_data_numeric[:,0], temporary_fill)

array([False, False, False, ..., False, False, False])

In [220]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum() #how many times we obtain true

0

✅ id column
___

# 🧐
#### Temporary Stats

In [224]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

In [223]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

In [222]:
temporary_stats[:, columns_numeric] #only interested in numeric data here

array([[  373332.  ,     1000.  ,     1000.  ,        6.  ,       31.42,        0.  ],
       [54015809.19,    15273.46,    15311.04,       16.62,      440.92,     3143.85],
       [68616519.  ,    35000.  ,    35000.  ,       28.99,     1372.97,    41913.62]])

- First row = min
- Second row = mean
- Third row = max

Funded_Amount is the only column for which we want to set the filler value equal to the minimum, since we assume the worst we assume that
<br>for those empty spaces the customer supplied the minimum amount possible

___

### Funded Amount (funded_amnt)

In [286]:
header_numeric[2]

'funded_amnt'

In [226]:
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [227]:
np.isin(loan_data_numeric[:,2], temporary_fill).sum()

500

In [287]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill,
                                  temporary_stats[0, columns_numeric[2]],      #min_value = first_row and second_column of the temporary_stats array of the numeric values only
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

array([35000., 30000., 15000., ..., 10000., 10000., 10000.])

In [229]:
np.isin(loan_data_numeric[:,2], temporary_fill).sum()

0

✅ All NaN values in in funded_amnt have been replaced with the min value
___

### Loaned Amount (loan_amnt), Interest Rate (int_rate), Installment (installment), Total Payment (total_paymnt)

In [231]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

For all these variables, NaN values will be replaced with the maximum value of each column, stored in temporary_stats

In [288]:
for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill,
                                      temporary_stats[2, columns_numeric[i]],
                                      loan_data_numeric[:,i])

In [289]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  ,       28.99,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  ,       28.99,      494.86,     1969.83],
       ...,
       [50415990.  ,    10000.  ,    10000.  ,       28.99,     1372.97,     2185.64],
       [46154151.  ,    35000.  ,    10000.  ,       16.55,      354.3 ,     3199.4 ],
       [66055249.  ,    10000.  ,    10000.  ,       28.99,      309.97,      301.9 ]])

✅ All NaN values have been replaced with the max value for respectiv e columns
___

### Excahnge Rate

# 🧐
We need the exchange rates of each loan application at the time of each application
<br>The file EUR-USD.csv containes the average monthly exchange rate for 2015

In [234]:
EUR_USD = np.genfromtxt('EUR-USD.csv', delimiter = ',', autostrip = True, dtype = str)
EUR_USD

array([['Open', 'High', 'Low', 'Close', 'Volume'],
       ['1.2098628282546997', '1.2098628282546997', '1.11055588722229', '1.1287955045700073', '0'],
       ['1.1287955045700073', '1.1484194993972778', '1.117680549621582', '1.1205360889434814',
        '0'],
       ['1.119795799255371', '1.1240400075912476', '1.0460032224655151', '1.0830246210098267',
        '0'],
       ['1.0741022825241089', '1.1247594356536865', '1.0521597862243652', '1.1114321947097778',
        '0'],
       ['1.1215037107467651', '1.145304799079895', '1.0821995735168457', '1.0960345268249512',
        '0'],
       ['1.095902442932129', '1.1428401470184326', '1.0888904333114624', '1.122296690940857', '0'],
       ['1.1134989261627197', '1.1219995021820068', '1.081270456314087', '1.0939244031906128',
        '0'],
       ['1.0969001054763794', '1.1705996990203857', '1.0850305557250977', '1.1340054273605347',
        '0'],
       ['1.1225990056991577', '1.1460003852844238', '1.1089695692062378', '1.1255937814712524

We will use the closing price as the exchange rate

In [235]:
#importing the closing value for each month (Jan-Dec) only as the average exchange rate for that month
EUR_USD = np.genfromtxt('EUR-USD.csv', delimiter = ',', autostrip = True, skip_header = True, usecols = 3)
EUR_USD

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

In [236]:
#Since we have months stored as numbers (1-12) in loan_data_strings[:,0]
loan_data_strings[:,0]
#0 where data was not provided

array([ 5,  0,  9, ...,  6,  4, 12])

In [290]:
exchange_rate = loan_data_strings[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)
    
#all zero values will be replaced by the annual mean
for i in range(1,13):                 
    exchange_rate = np.where(exchange_rate == 0,
                             np.mean(EUR_USD),
                             exchange_rate)
    
exchange_rate

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [241]:
exchange_rate.shape

(10000,)

In [242]:
loan_data_numeric.shape

(10000, 6)

In [293]:
#Because you can't add a 1 dimensional array to a 2 dimensional array, we have to reshape our exchange_rate array to the same dimensions
exchange_rate = np.reshape(exchange_rate, (10000, 1))

In [294]:
np.hstack((loan_data_numeric, exchange_rate))

array([[48010226.  ,    35000.  ,    35000.  , ...,     1184.86,     9452.96,        1.1 ],
       [57693261.  ,    30000.  ,    30000.  , ...,      938.57,     4679.7 ,        1.11],
       [59432726.  ,    15000.  ,    15000.  , ...,      494.86,     1969.83,        1.13],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     1372.97,     2185.64,        1.12],
       [46154151.  ,    35000.  ,    10000.  , ...,      354.3 ,     3199.4 ,        1.11],
       [66055249.  ,    10000.  ,    10000.  , ...,      309.97,      301.9 ,        1.09]])

In [295]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [296]:
#This results in a similar error as before. Trying to add a 0 dimensional scalar, the word 'exchange_rate' to a 1-dimensional array is not allowed.
#They have to have similar dimensions.
header_numeric = np.concatenate((header_numeric, 'exchange_rate'))
header_numeric

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 0 dimension(s)

In [297]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

✅ 
- Exchange rate values have been successfully extracted from the EUR-USD.csv file.
- The extracted values were stacked on to the loan_data_numeric column as 'exchange_rate'

# 🧐
#### From USD to EUR
Now that we have our average exchange-rate for each month of 2015 the following columns need to be converted from dollars to euro
- loan_amnt (loan_data_numeric[1])
- funded_amnt  (loan_data_numeric[2])
- installment  (loan_data_numeric[4])
- total_paymnt  (loan_data_numeric[5])

In [260]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate'],
      dtype='<U19')

In [261]:
#Declaring an array with numbers equal to the indices of the columns in the loan_data_numeric array that needs to undergo the currency conversion from dollars to euro
columns_dollar = np.array([1,2,4,5])

In [252]:
loan_data_numeric[:,6]

array([1.1 , 1.11, 1.13, ..., 1.12, 1.11, 1.09])

In [298]:
#As we iterate through each column in our dataset that has to be converted, every element gets divided by the equivelant row element in the exchange_rate column, thus giving us the converted value.
#The result is a 1 dimensional array with the converted values.
#Since we can't add a 1-dimensional array to a 2-dimensional array we use the np.reshape function to convert it (np.reshape(....,(10000,1))
#The converted array is then horizontally stacked to the loan_data_numeric array
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000, 1))))

In [299]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

In [258]:
loan_data_numeric.shape

(10000, 11)

✅ 
- All dollar values have been successfully converted to euros
- The results have been stacked to the loan_data_numeric dataset in their respective columns
- No origonal dollar value columns have been dropped at this point

#### Expanding the header

In [300]:
#iterating through the header_numeric array, again using our columns_dollar as the index range
#the value + '_EUR' get stored in a new array
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])
header_additional

array(['loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U15')

In [301]:
header_numeric = np.concatenate((header_numeric, header_additional))
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt', 'exchange_rate',
       'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'], dtype='<U19')

In [302]:
#Lets also change the names of the original dollar value columns to clearly denote their currency
#We only want the columns with dollar values, so once again we make use of our columns_dollar array to define the indices
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]]) #Each column_name is concatenated with _USD
header_numeric

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

In [266]:
#Improving this even further by rearranging the columns so that amounts in USD and Euros are paired for the same variable
columns_index_order  = [0,1,7,2,8,3,4,9,5,10,6]

In [303]:
header_numeric[columns_index_order]

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [304]:
#Updating column order in header
header_numeric = header_numeric[columns_index_order]

In [305]:
loan_data_numeric

array([[48010226.  ,    35000.  ,    35000.  , ...,    31933.3 ,     1081.04,     8624.69],
       [57693261.  ,    30000.  ,    30000.  , ...,    27132.46,      848.86,     4232.39],
       [59432726.  ,    15000.  ,    15000.  , ...,    13326.3 ,      439.64,     1750.04],
       ...,
       [50415990.  ,    10000.  ,    10000.  , ...,     8910.3 ,     1223.36,     1947.47],
       [46154151.  ,    35000.  ,    10000.  , ...,     8997.4 ,      318.78,     2878.63],
       [66055249.  ,    10000.  ,    10000.  , ...,     9145.8 ,      283.49,      276.11]])

In [306]:
#Updating loan_data_numeric column order to match header 
loan_data_numeric = loan_data_numeric[:,columns_index_order]
loan_data_numeric

array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
       [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
       [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
       [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
       [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]])

✅ 
- Order of dataset successfully updated to pair USD-EUR versions of the same variabel
___

### Interest Rate (int_rate)

In [307]:
header_numeric

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
      dtype='<U19')

In [308]:
loan_data_numeric[:,5]

array([13.33, 28.99, 28.99, ..., 28.99, 16.55, 28.99])

# 🧐
Values are stored in percentages. Since these are used in calculations it makes sense to **transform them to values between 1 and 0**.

In [309]:
#To achieve this divide the entire column by 100
loan_data_numeric[:,5]/100

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

In [310]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100
loan_data_numeric[:,5]

array([0.13, 0.29, 0.29, ..., 0.29, 0.17, 0.29])

✅ 
- Interest rate has been successfully transformed from percentage values to their equivelant value between 0 and 1
___

# 🔐
## Checkpoint 2: Numeric

In [311]:
#Creating a checkpoint with our previously declared function
checkpoint_numeric = checkpoint('Checkpoint-Numeric', header_numeric, loan_data_numeric)

In [312]:
checkpoint_numeric['header'], checkpoint_numeric['data']

(array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate'],
       dtype='<U19'),
 array([[48010226.  ,    35000.  ,    31933.3 , ...,     9452.96,     8624.69,        1.1 ],
        [57693261.  ,    30000.  ,    27132.46, ...,     4679.7 ,     4232.39,        1.11],
        [59432726.  ,    15000.  ,    13326.3 , ...,     1969.83,     1750.04,        1.13],
        ...,
        [50415990.  ,    10000.  ,     8910.3 , ...,     2185.64,     1947.47,        1.12],
        [46154151.  ,    35000.  ,    31490.9 , ...,     3199.4 ,     2878.63,        1.11],
        [66055249.  ,    10000.  ,     9145.8 , ...,      301.9 ,      276.11,        1.09]]))

___

## Creating the 'Complete' dataset
- Combine the preprocessed versions of the string and numeric matrices
- Sort
- Export to csv

#### Combining the datasets

In [313]:
#make sure that both datasets have the same shape (same number of rows)
checkpoint_strings['data'].shape

(10000, 6)

In [314]:
checkpoint_numeric['data'].shape

(10000, 11)

In [315]:
np.hstack((checkpoint_numeric['data'], checkpoint_strings['data'])).shape

(10000, 17)

In [316]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [317]:
loan_data

array([[48010226.  ,    35000.  ,    31933.3 , ...,       13.  ,        1.  ,        1.  ],
       [57693261.  ,    30000.  ,    27132.46, ...,        5.  ,        1.  ,        4.  ],
       [59432726.  ,    15000.  ,    13326.3 , ...,       10.  ,        1.  ,        4.  ],
       ...,
       [50415990.  ,    10000.  ,     8910.3 , ...,        5.  ,        1.  ,        1.  ],
       [46154151.  ,    35000.  ,    31490.9 , ...,       17.  ,        1.  ,        3.  ],
       [66055249.  ,    10000.  ,     9145.8 , ...,        4.  ,        0.  ,        3.  ]])

In [318]:
#final check to see if we missed an nan values
np.isnan(loan_data).sum()

0

In [319]:
np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

array(['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
       'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR', 'exchange_rate',
       'issue_date', 'loan_status', 'term_months', 'sub_grade', 'verification_status',
       'state_address'], dtype='<U19')

In [327]:
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

✅  Numeric and Strings dataset successfully combined

#### Sorting the new Dataset

In [320]:
#It makes sense to sort the array based on the id column
np.sort(loan_data[:,0])

array([  373332.,   575239.,   707689., ..., 68614880., 68615915., 68616519.])

In [321]:
np.argsort(loan_data[:,0]) #gives us the inices of each value in the ID column

array([2086, 4812, 2353, ..., 4935, 9388, 8415])

In [322]:
#We can use this to sort our entire dataset
loan_data[np.argsort(loan_data[:,0])]

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [323]:
#store values in dataset
loan_data = loan_data[np.argsort(loan_data[:,0])]
loan_data

array([[  373332.  ,     9950.  ,     9038.08, ...,       21.  ,        0.  ,        1.  ],
       [  575239.  ,    12000.  ,    10900.2 , ...,       25.  ,        1.  ,        2.  ],
       [  707689.  ,    10000.  ,     8924.3 , ...,       13.  ,        1.  ,        0.  ],
       ...,
       [68614880.  ,     5600.  ,     5121.65, ...,        8.  ,        1.  ,        1.  ],
       [68615915.  ,     4000.  ,     3658.32, ...,       10.  ,        1.  ,        2.  ],
       [68616519.  ,    21600.  ,    19754.93, ...,        3.  ,        0.  ,        2.  ]])

In [324]:
np.argsort(loan_data[:,0])

array([   0,    1,    2, ..., 9997, 9998, 9999])

✅ Combined dataset successfully sorted according to ID

#### Storing and exporting the new dataset

In [328]:
# Lets combine the header and dataset
np.vstack((header_full, loan_data))

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', ..., 'sub_grade', 'verification_status',
        'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', ..., '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', ..., '25.0', '1.0', '2.0'],
       ...,
       ['68614880.0', '5600.0', '5121.647851612413', ..., '8.0', '1.0', '1.0'],
       ['68615915.0', '4000.0', '3658.319894008867', ..., '10.0', '1.0', '2.0'],
       ['68616519.0', '21600.0', '19754.927427647883', ..., '3.0', '0.0', '2.0']], dtype='<U32')

In [329]:
loan_data = np.vstack((header_full, loan_data))

In [330]:
np.savetxt('loan-data-preprocessed.txt', 
            loan_data,
            fmt = '%s',
            delimiter = ',')

✅ Dataset successfully stored as loan-data-preprocessed.txt
___